In [ ]:
!pip install requests pandas numpy matplotlib seaborn scikit-learn tensorflow keras folium streamlit geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import json

In [ ]:
# Define OpenWeather API Key
OPENWEATHER_API_KEY = "bc4cba2ac5f4ba0f263b0829d0f41956"

# Fetch weather data
def get_weather(city="Delhi"):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather_info = {
            "Temperature": data["main"]["temp"],
            "Humidity": data["main"]["humidity"],
            "Weather": data["weather"][0]["description"]
        }
        return weather_info
    else:
        return {"error": "Failed to fetch weather data"}

# Test
print(get_weather("Mumbai"))


{'Temperature': 28.99, 'Humidity': 51, 'Weather': 'smoke'}


In [ ]:
TOMTOM_API_KEY = "WEpsAB2rzEvaGotcNIownr7uNbQYmr7f"

def get_traffic(lat=28.7041, lon=77.1025):
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/relative0/10/json?point={lat},{lon}&key={TOMTOM_API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        traffic_info = {
            "Current Speed": data["flowSegmentData"]["currentSpeed"],
            "Free Flow Speed": data["flowSegmentData"]["freeFlowSpeed"],
            "Traffic Level": data["flowSegmentData"]["confidence"]
        }
        return traffic_info
    else:
        return {"error": "Failed to fetch traffic data"}

# Test
print(get_traffic(28.7041, 77.1025))  # Example for Delhi


{'Current Speed': 18, 'Free Flow Speed': 29, 'Traffic Level': 0.992433}


In [ ]:
url = "https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv"
df = pd.read_csv(url)

# Show dataset
df.head()

,iata,airport,city,state,country,lat,long,cnt
0,ORD,Chicago O'Hare International,Chicago,IL,USA,41.979595,-87.904464,25129
1,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944,21925
2,DFW,Dallas-Fort Worth International,Dallas-Fort Worth,TX,USA,32.895951,-97.037200,20662
3,PHX,Phoenix Sky Harbor International,Phoenix,AZ,USA,33.434167,-112.008056,17290
4,DEN,Denver Intl,Denver,CO,USA,39.858408,-104.667002,13781


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Prepare dataset (Modify as per traffic dataset format)
df["traffic_index"] = df["cnt"]  # Assume "cnt" represents congestion level
df = df[["traffic_index", "lat", "long"]]  # Selecting relevant features

# Split dataset
X = df[["lat", "long"]]
y = df["traffic_index"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)

# Evaluate Model
error = mean_absolute_error(y_test, predictions)
print(f"Model MAE: {error}")


Model MAE: 2173.756666666667


In [ ]:
disease_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df_disease = pd.read_csv(disease_url)

# Show dataset
df_disease.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [ ]:
from sklearn.linear_model import LinearRegression

# Prepare dataset
df_disease = df_disease.iloc[:, -30:]  # Last 30 days data
df_disease["total_cases"] = df_disease.sum(axis=1)

# Train Model
X = df_disease.drop(columns=["total_cases"])
y = df_disease["total_cases"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_disease = LinearRegression()
model_disease.fit(X_train, y_train)

# Predict
future_cases = model_disease.predict(X_test)
print("Predicted Disease Cases:", future_cases[:5])


Predicted Disease Cases: [ 2.64364900e+06  5.24099100e+07  4.20900000e+04 -1.49011612e-08
  3.52026700e+06]


In [ ]:
!pip install streamlit

import streamlit as st

st.title("🚦 AI Traffic & Disease Prediction Dashboard")

city = st.text_input("Enter City for Weather & Traffic Data", "Delhi")

if st.button("Get Predictions"):
    weather = get_weather(city)
    traffic = get_traffic(28.7041, 77.1025)  # Change coordinates accordingly

    st.subheader("🌦 Weather Data")
    st.write(weather)

    st.subheader("🚗 Traffic Congestion")
    st.write(traffic)

    st.subheader("🦠 Disease Prediction")
    st.write("Predicted Future Cases:", future_cases[:5])


2025-03-18 13:30:11.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 13:30:11.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [14]:
!npm install -g vercel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
⠙⠹⠸npm warn deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
⠸⠼npm warn deprecated uuid@3.3.2: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 237 packages in 33s
⠋
⠋26 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [15]:
# Install Git on Colab
!apt install git -y

# Configure Git
!git config --global user.email "ankitgupta18k@gmail.com"
!git config --global user.name "ankitgpt18"

# Clone your GitHub repo (Replace with your actual GitHub repo URL)
!git clone https://github.com/ankitgpt18/AI-Traffic-Management

# Change directory to the cloned repo
%cd AI-Traffic-Management

# Copy all necessary Colab files to the GitHub repo
!cp /content/*.py .   # Copy all Python files
!cp /content/*.ipynb .  # Copy Jupyter Notebooks (if any)

# Add, Commit, and Push Changes to GitHub
!git add .
!git commit -m "Initial commit from Colab"
!git push origin main


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'AI-Traffic-Management'...
/content/AI-Traffic-Management
cp: cannot stat '/content/*.py': No such file or directory
cp: cannot stat '/content/*.ipynb': No such file or directory
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/ankitgpt18/AI-Traffic-Management'
